In [4]:
import guitarpro
import dadagp as dada
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import json

In [5]:
DADAGP_PATH = "acdc_ex.txt"
GP_PATH = "../util/test_acdc.gp5"
PREFIX = "./test_loops_out/acdc_test"

In [6]:
#snippet for extracting time signatures

song = guitarpro.parse(GP_PATH) #check if one time signature throughout, calculate min/max beat lengths based on it
timeSignatures = {}
for measure in song.tracks[0].measures[0:10]:
    signature = (measure.timeSignature.numerator, measure.timeSignature.denominator.value)
    if signature in timeSignatures.keys():
        timeSignatures[signature] += 1
    else:
        timeSignatures[signature] = 1
    print(measure.timeSignature.numerator, measure.timeSignature.denominator.value)
    print(measure.keySignature) #doesn't seem like this is accurate, just defaulting to CMaj. Could we auto extract key?
#what to do if time signature changes? could just use most prominent? or split up song?

4 4
KeySignature.CMajor
4 4
KeySignature.CMajor
4 4
KeySignature.CMajor
4 4
KeySignature.CMajor
4 4
KeySignature.CMajor
4 4
KeySignature.CMajor
4 4
KeySignature.CMajor
4 4
KeySignature.CMajor
4 4
KeySignature.CMajor
4 4
KeySignature.CMajor


In [ ]:
#create the json file for names of all the loop files

root_dir = "D:\Documents\DATA\DadaGP-Loops"
file_list = []
for subdir, dirs, files in os.walk(root_dir):
    short_subdir = subdir.replace(root_dir + "\\", "")
    for file in files:
        if file.endswith(".txt"):
            full_path = os.path.join(short_subdir, file)
            file_list.append(full_path)

#save that file list to json
root_dir = "D:\Documents\DATA\DadaGP-Loops"
path_json = os.path.join(root_dir, "_DadaGP_LOOPS_all_filenames.json")
with open(path_json, 'w') as f:
    json.dump(file_list, f)

In [7]:
#helper function for finding number of empty bars in a track

def calc_empty_tracks(test_file):
    song = guitarpro.parse(test_file)
    empty_tracks = 0
    total_tracks = len(song.tracks)
    for track in song.tracks:
        empty_bars = 0
        total_bars = len(track.measures)
        for measure in track.measures:
            non_rests = 0
            for beat in measure.voices[0].beats:
                for note in beat.notes:
                    if note.type != guitarpro.NoteType.rest:
                        non_rests = non_rests + 1
            if non_rests == 0:
                empty_bars = empty_bars + 1
        if empty_bars == total_bars:
            empty_tracks = empty_tracks + 1
    return empty_tracks, total_tracks

In [17]:
# report the number of empty bars across all tracks

root_path = "D:\Documents\DATA\DadaGP-Loops"
allfiles_path = os.path.join(root_path,"file_list.json" )
with open(allfiles_path, "r") as f:
    allfiles =  json.load(f)

n_files = len(allfiles)
total_tracks = 0
empty_tracks = 0
for fidx in range(n_files):   
    fn = allfiles[fidx].replace(".txt", ".gp5")
    file = os.path.join(root_path, fn)
    empty, total = calc_empty_tracks(file)
    total_tracks = total_tracks + total
    empty_tracks = empty_tracks + empty

print("{} empty tracks out of {} total tracks".format(empty_tracks, total_tracks))

0 empty tracks out of 140179 total tracks


In [9]:
35003.0 / 176703.0 #percentage of empty tracks

0.19808944952830457

In [15]:
#get max number of tokens in dataset 

root_path = "D:\Documents\DATA\DadaGP-Loops"
allfiles_path = os.path.join(root_path,"file_list.json" )
with open(allfiles_path, "r") as f:
    allfiles =  json.load(f)

n_files = len(allfiles)
max_lines = 0
for fidx in range(n_files):   
    fn = allfiles[fidx]
    file = os.path.join(root_path, fn)
    with open(file) as f:
        num_lines = len(f.read().splitlines())
        if num_lines > max_lines:
            max_lines = num_lines
print("{} maximum token length".format(max_lines))

19629 maximum token length


In [13]:
9817.0 / 512 #group size for dataset with 4 repeats

19.173828125

In [16]:
19629.0 / 512 #group size for dataset with 8 repeats

38.337890625

In [ ]:
#extract 25 "real" loops following genre distribution
#each loop should be 4 bars long (should we restrict to 4/4 time signature?)
#TODO: how to get the genre distribution? we have it as JSON files but #2 genre is "unknown." Should we filter out and only use top few genres? and do this in training too?
#for generation, was thinking for each real example we generate artifical one with the same tempo and starting beat (since a primer is needed)
#should we include genre/artist tags in the artificial generation and does this matter? 
#try prompting with an open repeat symbol, could make the training dataset have the repeat
#training only on guitar parts?
#average number of instruments per track? Could be predominantly only guitar or something
#try training on different checkpoints other than 200
#try changing the group size